#### Trong bài tập này, bạn sẽ sử dụng dữ liệu này để trả về số lượng xe đạp trung bình có sẵn theo giờ trong một tuần (22 tháng 2 đến 28 tháng 2) cho các đài chỉ nằm trong khu vực San Jose. 

In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, HiveContext
sc = SparkContext.getOrCreate()
hiveContext = HiveContext(sc)
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

Create an RDD named stations/status/

In [5]:
stations = sc.textFile('data/bike-share/stations.csv')
status = sc.textFile('data/bike-share/status.csv')

In [4]:
status2 = status.map(lambda x: x.split(','))
status2.first()

['196', '242', '3', '881250949']

In [4]:
status2 = status.map(lambda x: x.split(',')) \
.map(lambda x: (x[0], x[1], x[2], x[3].replace('"',''))) \
.map(lambda x: (x[0], x[1], x[2], x[3].split(' '))) 
status2.first()

('10', '9', '6', ['2015-02-28', '23:59:01'])

In [5]:
status2 = status.map(lambda x: x.split(',')) \
.map(lambda x: (x[0], x[1], x[2], x[3].replace('"',''))) \
.map(lambda x: (x[0], x[1], x[2], x[3].split(' '))) \
.map(lambda x: (x[0], x[1], x[2], x[3][0].split('-'), x[3][1].split(':'))) 
status2.first()

('10', '9', '6', ['2015', '02', '28'], ['23', '59', '01'])

In [6]:
status2 = status.map(lambda x: x.split(',')) \
.map(lambda x: (x[0], x[1], x[2], x[3].replace('"',''))) \
.map(lambda x: (x[0], x[1], x[2], x[3].split(' '))) \
.map(lambda x: (x[0], x[1], x[2], x[3][0].split('-'), x[3][1].split(':'))) \
.map(lambda x: (int(x[0]), int(x[1]), int(x[3][0]), int(x[3][1]), int(x[3][2]), int(x[4][0])))
status2.first()

(10, 9, 2015, 2, 28, 23)

Lấy từ 22-28/2

In [7]:
status3 = status2.filter(lambda x: x[2]==2015 and \
 x[3]==2 and \
 x[4]>=22) \
 .map(lambda x: (x[0], x[1], x[5]))
status3.first()

(10, 9, 23)

Lọc tập dữ liệu trạm 'San Jose'

In [8]:
stations2 = stations.map(lambda x: x.split(','))
stations2.first()

['2',
 'San Jose Diridon Caltrain Station',
 '37.329732',
 '-121.901782',
 '27',
 'San Jose',
 '8/6/2013']

In [9]:
stations2 = stations.map(lambda x: x.split(',')) \
 .filter(lambda x: x[5] == 'San Jose')
stations2.first()

['2',
 'San Jose Diridon Caltrain Station',
 '37.329732',
 '-121.901782',
 '27',
 'San Jose',
 '8/6/2013']

In [10]:
stations2 = stations.map(lambda x: x.split(',')) \
 .filter(lambda x: x[5] == 'San Jose') \
 .map(lambda x: (int(x[0]), x[1]))
stations2.first()

(2, 'San Jose Diridon Caltrain Station')

Chuyển đổi cả hai RDD thành RDD cặp key/value để chuẩn bị cho một phép join hoạt động

In [11]:
status_kv = status3.keyBy(lambda x: x[0])
stations_kv = stations2.keyBy(lambda x: x[0])

Nối cặp key/value status_kv RDD vào key/value stations_kv ghép nối RDD bằng các khóa của chúng (station_id)

In [12]:
joined = status_kv.join(stations_kv)
joined.first()

(12, ((12, 10, 23), (12, 'SJSU 4th at San Carlos')))

In [13]:
cleaned = joined.map(lambda x: (x[0], x[1][0][1], x[1][0][2], x[1][1][1]))
cleaned.first()

(12, 10, 23, 'SJSU 4th at San Carlos')

Tạo một cặp key/value với khóa dưới dạng một bộ bao gồm trạm đặt tên và giờ, sau đó tính giá trị trung bình theo từng giờ cho mỗi trạm:

In [14]:
avgbyhour = cleaned.keyBy(lambda x: (x[3],x[2]))
avgbyhour.first()

(('SJSU 4th at San Carlos', 23), (12, 10, 23, 'SJSU 4th at San Carlos'))

In [15]:
avgbyhour = cleaned.keyBy(lambda x: (x[3],x[2])) \
 .mapValues(lambda x: (x[1], 1)) 
avgbyhour.take(5)

[(('SJSU 4th at San Carlos', 23), (10, 1)),
 (('SJSU 4th at San Carlos', 23), (10, 1)),
 (('SJSU 4th at San Carlos', 23), (10, 1)),
 (('SJSU 4th at San Carlos', 23), (10, 1)),
 (('SJSU 4th at San Carlos', 23), (10, 1))]

In [16]:
avgbyhour = cleaned.keyBy(lambda x: (x[3],x[2])) \
 .mapValues(lambda x: (x[1], 1)) \
 .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
avgbyhour.take(5)

[(('SJSU 4th at San Carlos', 21), (4833, 420)),
 (('SJSU 4th at San Carlos', 17), (5071, 420)),
 (('SJSU 4th at San Carlos', 13), (5250, 420)),
 (('SJSU 4th at San Carlos', 9), (5323, 420)),
 (('SJSU 4th at San Carlos', 5), (5030, 420))]

In [17]:
avgbyhour = cleaned.keyBy(lambda x: (x[3],x[2])) \
 .mapValues(lambda x: (x[1], 1)) \
 .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
 .mapValues(lambda x: (x[0]/x[1]))
avgbyhour.take(5)

[(('SJSU 4th at San Carlos', 21), 11.507142857142858),
 (('SJSU 4th at San Carlos', 17), 12.073809523809524),
 (('SJSU 4th at San Carlos', 13), 12.5),
 (('SJSU 4th at San Carlos', 9), 12.673809523809524),
 (('SJSU 4th at San Carlos', 5), 11.976190476190476)]

Tìm 10 điểm trung bình hàng đầu theo trạm và giờ bằng cách sử dụng sortBy()

In [18]:
topavail = avgbyhour.keyBy(lambda x: x[1]) \
 .sortByKey(ascending=False)
topavail.take(10)

[(16.590476190476192,
  (('San Jose Diridon Caltrain Station', 17), 16.590476190476192)),
 (16.492857142857144,
  (('San Jose Diridon Caltrain Station', 7), 16.492857142857144)),
 (16.34285714285714,
  (('San Jose Diridon Caltrain Station', 6), 16.34285714285714)),
 (16.21904761904762,
  (('San Jose Diridon Caltrain Station', 18), 16.21904761904762)),
 (15.64047619047619,
  (('San Jose Diridon Caltrain Station', 19), 15.64047619047619)),
 (15.516666666666667,
  (('San Jose Diridon Caltrain Station', 22), 15.516666666666667)),
 (15.445238095238095,
  (('San Jose Diridon Caltrain Station', 0), 15.445238095238095)),
 (15.416666666666666,
  (('San Jose Diridon Caltrain Station', 20), 15.416666666666666)),
 (15.392857142857142,
  (('San Jose Diridon Caltrain Station', 1), 15.392857142857142)),
 (15.383333333333333,
  (('San Jose Diridon Caltrain Station', 4), 15.383333333333333))]

In [19]:
topavail = avgbyhour.keyBy(lambda x: x[1]) \
 .sortByKey(ascending=False) \
 .map(lambda x: (x[1][0][0], x[1][0][1], x[0]))
topavail.take(10)

[('San Jose Diridon Caltrain Station', 17, 16.590476190476192),
 ('San Jose Diridon Caltrain Station', 7, 16.492857142857144),
 ('San Jose Diridon Caltrain Station', 6, 16.34285714285714),
 ('San Jose Diridon Caltrain Station', 18, 16.21904761904762),
 ('San Jose Diridon Caltrain Station', 19, 15.64047619047619),
 ('San Jose Diridon Caltrain Station', 22, 15.516666666666667),
 ('San Jose Diridon Caltrain Station', 0, 15.445238095238095),
 ('San Jose Diridon Caltrain Station', 20, 15.416666666666666),
 ('San Jose Diridon Caltrain Station', 1, 15.392857142857142),
 ('San Jose Diridon Caltrain Station', 4, 15.383333333333333)]